In [12]:
from casadi import *

# Paper Cascaded Nonlinear MPC for Realtime Quadrotor Position Tracking. Schlagenhauf

# Define system, static/kinematic/dynamic model. Typically Differential (Algebraic) equations

# Expression graphs represent a computation in the computer memory. SX, MX


Ix = 0.0000166  #Moment of inertia around p_WB_W_x-axis, source: Julian Förster's ETH Bachelor Thesis
Iy = 0.0000167  #Moment of inertia around p_WB_W_y-axis, source: Julian Förster's ETH Bachelor Thesis
Iz = 0.00000293 #Moment of inertia around p_WB_W_z-axis, source: Julian Förster's ETH Bachelor Thesis
m = 0.029 #mass of Crazyflie 2.1
g = 9.81

Nx = 12
Nu = 4
Nhoriz = 10

# x(t) = [x, y, z, phi, theta, psi, x_dot, y_dot, z_dot, phi_dot, ˙theta_dot, psi_dot]T
X = MX.sym("x",Nx)
U = MX.sym("u",Nu)


x_pos = X[0]  # x-position
y = X[1]  # y-position
z = X[2]  # z-position
phi = X[3]  # phi-angle, Euler angles
theta = X[4]  # theta-angle, Euler angles
psi = X[5]  # psi-angle, Euler angles
x_pos_dot = X[6]  # x velocity
y_dot = X[7]  # y velocity
z_dot = X[8]  # z velocity
phi_dot = X[9]  # phi_dot, angular velocity
theta_dot = X[10]  # theta_dot
psi_dot = X[11]  # psi-dot

thrust = U[0]
tau_phi = U[1]
tau_theta = U[2]
tau_psi = U[3]


# x˙(t) = [x_dot, y_dot, z_dot, phi_dot, ˙theta_dot, psi_dot, x_ddot, y_ddot, z_ddot, phi_ddot, ˙theta_ddot, psi_ddot]T


x_pos_ddot = (cos(phi) * sin(phi) * cos(psi) + sin(phi) * sin(phi))  * thrust / m
y_ddot = (cos(phi) * sin(phi) * cos(psi) - sin(phi) * sin(phi))  * thrust / m
z_ddot = -g + (cos(phi) * cos(theta)) * thrust / m
phi_ddot = theta_dot * psi_dot * (Iy - Iz) / (Ix) + tau_phi / Ix
theta_ddot = phi_dot * psi_dot * (Iz - Ix) / (Iy) + tau_phi / Iy
psi_ddot = theta_dot * phi_dot * (Ix - Iy) / (Iz) + tau_phi / Iz

x_dot = vertcat(x_pos_dot, y_dot, z_dot, phi_dot, theta_dot, psi_dot, x_pos_ddot, y_ddot, z_ddot, phi_ddot, theta_ddot, psi_ddot)

f = Function('f', [X, U], [x_dot], ['x','u'],['x_dot'])

f

In [13]:
def rk4_step(f, X, U, dt):
    k1 = f(X, U)
    k2 = f(X + dt/2 * k1, U)
    k3 = f(X + dt/2 * k2, U)
    k4 = f(X + dt * k3, U)
    X_next = X + dt/6 * (k1 + 2*k2 + 2*k3 + k4)
    return X_next


In [15]:
# ---- objective          ---------


Q = diag(MX([100, 100, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
R = diag(MX([10.0, 10.0, 10.0, 10.0]))

# Add the objective function to the optimization problem
objective = 0  # Initialize the objective function

# Loop over the control horizon
for k in range(Nhoriz):
    # State tracking cost
    state_error = X[:, k] - Xref[:, k]  # State deviation
    objective += state_error.T @ Q @ state_error  # Quadratic cost for state deviation

    # Control effort cost
    if k < Nhoriz - 1:  # No control input for the last stage
        control_error = U[:, k]  # Assuming zero as the reference for control inputs
        objective += control_error.T @ R @ control_error  # Quadratic cost for control effort

# Terminal cost
terminal_error = X[:, Nhoriz] - Xref[:, Nhoriz]  # Terminal state deviation
objective += terminal_error.T @ Q @ terminal_error  # Quadratic cost for terminal state deviation

# Solve/deploy problem
# Numerical backends, 3rd-party solvers
# Ipopt

opti.minimize(objective)  # Set the objective in the optimization problem